In [7]:
import pulp
import ipywidgets as widgets

class App:
    def __init__(self):
        # Параметры оптимизации
        self._total_volume, total_box = self._create_editbar(1000, 'Total Volume (тонн)')
        self._octane, octane_box = self._create_editbar(80, 'Min Octane Number')
        self._sulfur, sulfur_box = self._create_editbar(0.05, 'Max Sulfur Content (%)')
        self._lead, lead_box = self._create_editbar(0.01, 'Max Lead Content (г/л)')

        # Добавить обработчики событий на изменение значений
        self._total_volume.observe(self._on_change, names='value')
        self._octane.observe(self._on_change, names='value')
        self._sulfur.observe(self._on_change, names='value')
        self._lead.observe(self._on_change, names='value')
        
        self._text_result, result_box = self._create_textarea('Results')

        # Интерфейс приложения
        app_container = widgets.VBox([
            total_box,
            octane_box,
            sulfur_box,
            lead_box,
            result_box
        ], layout=widgets.Layout(width='700px'))

        self.container = widgets.VBox([
            widgets.HTML('<h1>Fuel Optimization App</h1>'),
            app_container
        ])

        self._update_app()

    def _create_editbar(self, initial_val, label):
        eb_label = widgets.Label(label)
        eb = widgets.BoundedFloatText(value=initial_val, max=10000000)
        eb_box = widgets.HBox([eb_label, eb])
        return eb, eb_box

    def _create_textarea(self, text):
        result_label = widgets.Label('Results:')
        result_text = widgets.Textarea(
            value=text,
            layout=widgets.Layout(width='550px', height='300px')
        )
        result_box = widgets.VBox([result_label, result_text])
        return result_text, result_box

    def _on_change(self, _):
        self._update_app()

    def _update_app(self):
        total_volume = self._total_volume.value
        octane = self._octane.value
        sulfur = self._sulfur.value
        lead = self._lead.value

        result = self.run_optimization(total_volume, octane, sulfur, lead)
        self._text_result.value = result

    def run_optimization(self, total_volume, octane, sulfur, lead):
        model = pulp.LpProblem("Minimize_Cost_of_Production", pulp.LpMinimize)

        x = pulp.LpVariable.dicts("x", [1, 2, 3, 4], lowBound=0, cat='Continuous')

        costs = [10.5, 11, 12, 14]
        octane_numbers = [65, 75, 80, 95]
        sulfur_content = [0.07, 0.06, 0.04, 0.03]
        lead_content = [0.011, 0.010, 0.009, 0.009]
        resources = [700, 200, 800, 300]

        # Целевая функция
        model += pulp.lpSum([costs[i] * x[i + 1] for i in range(4)]) / total_volume, "Cost per Liter"

        # Ограничения
        model += pulp.lpSum([x[i + 1] for i in range(4)]) == total_volume, "Total Volume"
        model += pulp.lpSum([octane_numbers[i] * x[i + 1] for i in range(4)]) / total_volume >= octane, "Octane Number"
        model += pulp.lpSum([sulfur_content[i] * x[i + 1] for i in range(4)]) / total_volume <= sulfur, "Sulfur Content"
        model += pulp.lpSum([lead_content[i] * x[i + 1] for i in range(4)]) / total_volume <= lead, "Lead Content"
        for i in range(4):
            model += x[i + 1] <= resources[i], f"Resource limit for Component {i + 1}"

        model.solve()

        if pulp.LpStatus[model.status] != "Optimal":
            return "Решение отсутствует"

        result = []
        result.append("Статус: " + pulp.LpStatus[model.status])
        for i in range(4):
            result.append(f"Компонент {i + 1}: {x[i + 1].varValue} тонн")

        actual_octane = sum([octane_numbers[i] * x[i + 1].varValue for i in range(4)]) / total_volume
        actual_sulfur = sum([sulfur_content[i] * x[i + 1].varValue for i in range(4)]) / total_volume
        actual_lead = sum([lead_content[i] * x[i + 1].varValue for i in range(4)]) / total_volume

        result.append(f"Фактическое октановое число: {actual_octane}")
        result.append(f"Фактическое содержание серы: {actual_sulfur}")
        result.append(f"Фактическое содержание свинца: {actual_lead}")
        result.append("Общая себестоимость за литр: " + str(pulp.value(model.objective)) + " тыс. руб.")

        return '\n'.join(result)

app = App()
app.container


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/dshishov/miniforge3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0c/6_96gth10j72ydgstzw9m81r0000gp/T/0b04aac0ed1b4e51831af6d382d2158e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0c/6_96gth10j72ydgstzw9m81r0000gp/T/0b04aac0ed1b4e51831af6d382d2158e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 38 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 8 rows, 4 columns and 20 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-4) rows, 4 (0) columns and 16 (-4) elements
0  Obj 0 Primal inf 1842.1053 (2)
5  Obj 11.933333
Optimal - objective value 11.933333
After Postsolve, objective 11.933333, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 11.93333333 - 5 iterations time 0.002, Presolve 0.00
Option for printin

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/dshishov/miniforge3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0c/6_96gth10j72ydgstzw9m81r0000gp/T/a1d49d5a7ea946cca12816696a8acaf1-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0c/6_96gth10j72ydgstzw9m81r0000gp/T/a1d49d5a7ea946cca12816696a8acaf1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 38 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 8 rows, 4 columns and 20 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-4) rows, 4 (0) columns and 16 (-4) elements
0  Obj 1.4182783 Primal inf 1782.2818 (2)
2  Obj 14.7 Primal inf 4100 (2)
Primal infeasible - objective value 14.7
Presolved problem not optimal, resolve after postsolve
After Postsolve, objective 14.7, infeasibilities - dual 0 (0), primal 4100 (2)
PrimalIn